In [1]:
!pip install scikit-learn==0.23.2
!pip install pycaret[full]
from pycaret.classification import *

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np

In [3]:
train = pd.read_csv('/content/drive/MyDrive/MLIA/Data/titanic/train.csv')
test = pd.read_csv('/content/drive/MyDrive/MLIA/Data/titanic/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/MLIA/Data/titanic/gender_submission.csv')
all_data = pd.concat([train, test], axis=0)

In [4]:
all_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1, inplace = True)
all_data['Age'].fillna('24', inplace=True)  #use mode to fillna(age)
all_data['Fare'].fillna(all_data['Fare'].mean(), inplace=True)
all_data['Embarked'].fillna('S', inplace=True)
std = StandardScaler()
all_data['Pclass'] = std.fit_transform(np.array(all_data['Pclass']).reshape(-1,1))
all_data['Age'] = std.fit_transform(np.array(all_data['Age']).reshape(-1,1))
all_data['SibSp'] = std.fit_transform(np.array(all_data['SibSp']).reshape(-1,1))
all_data['Parch'] = std.fit_transform(np.array(all_data['Parch']).reshape(-1,1))
all_data['Fare'] = std.fit_transform(np.array(all_data['Fare']).reshape(-1,1))
all_data = pd.get_dummies(all_data, drop_first=True)
print(all_data.head())

   Survived    Pclass       Age     SibSp  Parch      Fare  Sex_male  \
0       0.0  0.841916 -0.511722  0.481288 -0.445 -0.503595         1   
1       1.0 -1.546098  0.710389  0.481288 -0.445  0.734503         0   
2       1.0  0.841916 -0.206195 -0.479087 -0.445 -0.490544         0   
3       1.0 -1.546098  0.481243  0.481288 -0.445  0.382925         0   
4       0.0  0.841916  0.481243 -0.479087 -0.445 -0.488127         1   

   Embarked_Q  Embarked_S  
0           0           1  
1           0           0  
2           0           1  
3           0           1  
4           0           1  


In [11]:
train = all_data.iloc[:891, :]
test = all_data.iloc[891:, :]
test.drop(['Survived'], axis = 1, inplace=True)
train['Survived'] = train['Survived'].astype(int)
print(train.head())

   Survived    Pclass       Age     SibSp  Parch      Fare  Sex_male  \
0         0  0.841916 -0.511722  0.481288 -0.445 -0.503595         1   
1         1 -1.546098  0.710389  0.481288 -0.445  0.734503         0   
2         1  0.841916 -0.206195 -0.479087 -0.445 -0.490544         0   
3         1 -1.546098  0.481243  0.481288 -0.445  0.382925         0   
4         0  0.841916  0.481243 -0.479087 -0.445 -0.488127         1   

   Embarked_Q  Embarked_S  
0           0           1  
1           0           0  
2           0           1  
3           0           1  
4           0           1  


In [32]:
ntrain = pd.concat([x_train, y_train], axis=1)
print(ntrain)

       Pclass       Age     SibSp     Parch      Fare  Sex_male  Embarked_Q  \
368  0.841916 -0.358959 -0.479087 -0.445000 -0.493927         0           1   
542  0.841916 -1.351924  3.362411  1.866526 -0.039066         0           0   
49   0.841916 -0.817250  0.481288 -0.445000 -0.299608         0           0   
667  0.841916 -0.358959 -0.479087 -0.445000 -0.493444         1           0   
297 -1.546098 -2.039362  0.481288  1.866526  2.286476         0           0   
..        ...       ...       ...       ...       ...       ...         ...   
650  0.841916 -0.358959 -0.479087 -0.445000 -0.491108         1           0   
744  0.841916  0.175715 -0.479087 -0.445000 -0.490544         1           0   
265 -0.352091  0.557625 -0.479087 -0.445000 -0.440755         1           0   
337 -1.546098  0.939535 -0.479087 -0.445000  1.956811         0           0   
464  0.841916 -0.358959 -0.479087 -0.445000 -0.488127         1           0   

     Embarked_S  Survived  
368           0        

In [33]:
s = setup(data=ntrain,
          session_id=1,
          target='Survived',
          train_size=0.99,
          fold_strategy='stratifiedkfold',
          fold=10,
          fold_shuffle=True,
          silent=True,
          ignore_low_variance=True,
          remove_multicollinearity = True,
          normalize = True,
          normalize_method = 'robust')

,Description,Value
0,session_id,1
1,Target,Survived
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(712, 9)"
5,Missing Values,False
6,Numeric Features,8
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=False, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='Survived',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_st...
                ('dummy', Dummify(target='Survived')),
                ('fix_perfect', Remove_100(target='Survived')),
                ('clean_names', Clean_Colum_Names()),
 

In [34]:
model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8268,0.8758,0.7135,0.8168,0.7587,0.6247,0.6309,0.103
catboost,CatBoost Classifier,0.8196,0.8733,0.6875,0.8180,0.7436,0.6063,0.6147,1.299
lightgbm,Light Gradient Boosting Machine,0.8153,0.8728,0.7209,0.7817,0.7484,0.6031,0.6059,0.122
rf,Random Forest Classifier,0.8083,0.8683,0.7283,0.7621,0.7431,0.5905,0.5925,0.519
ada,Ada Boost Classifier,0.8069,0.8440,0.7472,0.7506,0.7466,0.5908,0.5932,0.106
et,Extra Trees Classifier,0.8041,0.8416,0.7174,0.7568,0.7344,0.5798,0.5820,0.461
xgboost,Extreme Gradient Boosting,0.8040,0.8651,0.7063,0.7670,0.7336,0.5791,0.5821,0.503
lr,Logistic Regression,0.7983,0.8560,0.7132,0.7517,0.7289,0.5688,0.5722,0.461
dt,Decision Tree Classifier,0.7956,0.7870,0.7323,0.7350,0.7326,0.5673,0.5684,0.016
ridge,Ridge Classifier,0.7911,0.0000,0.6909,0.7491,0.7151,0.5510,0.5555,0.015


INFO:logs:create_model_container: 16
INFO:logs:master_model_container: 16
INFO:logs:display_container: 2
INFO:logs:GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=1, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
INFO:logs:compare_models() succesfully completed......................................


In [35]:
gbc = create_model('gbc')

df_pred = predict_model(gbc, x_test)
y_pred = df_pred.loc[:, ['Label']]
y_pred = y_pred.astype(int)
submission['Survived'] = y_pred
submission['Survived'] = submission['Survived'].astype(int)
submission.to_csv('/content/drive/MyDrive/MLIA/Data/titanic/submission.csv', index=False)
print(submission['Survived'].dtype)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8310,0.8729,0.7037,0.8261,0.7600,0.6308,0.6357
1,0.7746,0.8388,0.5926,0.7619,0.6667,0.5004,0.5095
2,0.8873,0.9259,0.7778,0.9130,0.8400,0.7539,0.7597
3,0.7746,0.8603,0.7407,0.6897,0.7143,0.5286,0.5296
4,0.8286,0.8811,0.8148,0.7586,0.7857,0.6432,0.6443
5,0.7857,0.8445,0.7037,0.7308,0.7170,0.5447,0.5449
6,0.8714,0.9707,0.7407,0.9091,0.8163,0.7190,0.7279
7,0.8714,0.9035,0.7778,0.8750,0.8235,0.7230,0.7261
8,0.8000,0.8303,0.6296,0.8095,0.7083,0.5597,0.5700


INFO:logs:create_model_container: 17
INFO:logs:master_model_container: 17
INFO:logs:display_container: 3
INFO:logs:GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=1, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
INFO:logs:create_model() succesfully completed......................................


In [22]:
from sklearn.model_selection import train_test_split
x = train.iloc[:, 1:]
y = train.iloc[:, 0]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=1)

In [36]:
from sklearn.metrics import classification_report
df_pred = predict_model(gbc, x_test)
pred = df_pred.loc[:, ['Label']]
print(classification_report(y_test, pred))

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=1, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False), probability_threshold=None, encoded_labels=False, drift_report=False, raw_score=False, round=4, verbose=True, ml_usecase=MLUsecase.CLASSIFICATION, display=None, drift_kwargs=None)
INFO:logs:Checking exceptions
INFO:logs:Preloading l

              precision    recall  f1-score   support

           0       0.85      0.90      0.88       110
           1       0.83      0.75      0.79        69

    accuracy                           0.84       179
   macro avg       0.84      0.83      0.83       179
weighted avg       0.84      0.84      0.84       179



In [40]:
print(y_train.value_counts())
print(y_test.value_counts())

0    439
1    273
Name: Survived, dtype: int64
0    110
1     69
Name: Survived, dtype: int64


In [56]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=1)
x_train_sm, y_train_sm = sm.fit_resample(x_train, y_train)
x_test_sm, y_test_sm = sm.fit_resample(x_test, y_test)
x_train_sm = pd.DataFrame(x_train_sm)
y_train_sm = pd.DataFrame(y_train_sm)
x_test_sm = pd.DataFrame(x_test_sm)
y_test_sm = pd.DataFrame(y_test_sm)

In [57]:
print(y_train_sm.value_counts())
print(y_test_sm.value_counts())

Survived
0           439
1           439
dtype: int64
Survived
0           110
1           110
dtype: int64


In [59]:
train_sm = pd.concat([x_train_sm, y_train_sm], axis=1)
print(train_sm)

       Pclass       Age     SibSp     Parch      Fare  Sex_male  Embarked_Q  \
0    0.841916 -0.358959 -0.479087 -0.445000 -0.493927         0           1   
1    0.841916 -1.351924  3.362411  1.866526 -0.039066         0           0   
2    0.841916 -0.817250  0.481288 -0.445000 -0.299608         0           0   
3    0.841916 -0.358959 -0.479087 -0.445000 -0.493444         1           0   
4   -1.546098 -2.039362  0.481288  1.866526  2.286476         0           0   
..        ...       ...       ...       ...       ...       ...         ...   
873 -1.546098 -0.613857 -0.479087  1.353589  0.684750         0           0   
874 -1.546098 -0.273723  0.481288 -0.445000  1.125551         0           0   
875 -1.546098  0.293058 -0.479087 -0.445000 -0.053403         1           0   
876 -1.546098  0.481243  0.481288 -0.445000  0.493473         0           0   
877 -1.546098 -0.358959 -0.479087 -0.445000  0.988585         0           0   

     Embarked_S  Survived  
0             0        

In [60]:
s = setup(data=train_sm,
          session_id=1,
          target='Survived',
          train_size=0.99,
          fold_strategy='stratifiedkfold',
          fold=10,
          fold_shuffle=True,
          silent=True,
          ignore_low_variance=True,
          remove_multicollinearity = True,
          normalize = True,
          normalize_method = 'robust')

,Description,Value
0,session_id,1
1,Target,Survived
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(878, 9)"
5,Missing Values,False
6,Numeric Features,8
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=False, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='Survived',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_st...
                ('dummy', Dummify(target='Survived')),
                ('fix_perfect', Remove_100(target='Survived')),
                ('clean_names', Clean_Colum_Names()),
 

In [61]:
model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8424,0.9016,0.8279,0.8543,0.8399,0.6847,0.6866,0.203
xgboost,Extreme Gradient Boosting,0.8401,0.9105,0.8349,0.8453,0.8389,0.6802,0.6821,0.435
lightgbm,Light Gradient Boosting Machine,0.8378,0.9105,0.8462,0.8342,0.8388,0.6755,0.6778,0.148
rf,Random Forest Classifier,0.8343,0.9111,0.8393,0.8323,0.8342,0.6686,0.6711,0.555
catboost,CatBoost Classifier,0.8308,0.9119,0.8118,0.8472,0.8278,0.6617,0.6643,1.889
et,Extra Trees Classifier,0.8275,0.8872,0.8325,0.8249,0.8273,0.6549,0.6571,0.496
dt,Decision Tree Classifier,0.8251,0.8289,0.8348,0.8194,0.8255,0.6502,0.6526,0.018
ada,Ada Boost Classifier,0.8205,0.8749,0.8254,0.8189,0.8217,0.6409,0.6417,0.127
qda,Quadratic Discriminant Analysis,0.8009,0.8573,0.7863,0.8114,0.7972,0.6019,0.6041,0.019
knn,K Neighbors Classifier,0.7986,0.8628,0.8392,0.7769,0.8062,0.5971,0.6003,0.122


INFO:logs:create_model_container: 16
INFO:logs:master_model_container: 16
INFO:logs:display_container: 2
INFO:logs:GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=1, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
INFO:logs:compare_models() succesfully completed......................................


In [68]:
from sklearn.metrics import classification_report
df_pred = predict_model(gbc, test)
pred = df_pred.loc[:, ['Label']]


INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=1, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False), probability_threshold=None, encoded_labels=False, drift_report=False, raw_score=False, round=4, verbose=True, ml_usecase=MLUsecase.CLASSIFICATION, display=None, drift_kwargs=None)
INFO:logs:Checking exceptions
INFO:logs:Preloading l

In [69]:
submission['Survived'] = pred
submission['Survived'] = submission['Survived'].astype(int)
submission.to_csv('/content/drive/MyDrive/MLIA/Data/titanic/submission.csv', index=False)